In [1]:
from gazpacho import get, Soup

In [2]:
url = 'https://www.capfriendly.com/'
html = get(url)

In [3]:
soup = Soup(html)

In [4]:
table = soup.find('table', {'id': 'ich'})

In [5]:
str(table)[:100]

'<table id="ich" class="sortablex tblcf index tbl sortable"><thead><tr class="column_head"><th class='

In [6]:
trs = table.find('tr', {'class': 'tmx'})

In [7]:
tr = trs[0]

In [8]:
tr.find('a', mode='first').text

'Arizona Coyotes'

In [9]:
print(tr)

<tr class="odd tmx_2" data-sort-f="42359999" data-sort-d="29536625" data-sort-g="6100000" data-sort-inj2="5275000" data-sort-tch="85216068" data-sort-team="Arizona Coyotes" data-sort-rs="23" data-sort-spc="45" data-sort-inj="1" data-sort-ch="85089798" data-sort-ch2="3589798" data-sort-ch3="-81500000"><td class="tmx left" data-team="2" data-label="TEAM ▾"><span><img class="im_mid" style="height:15px;width:15px;margin-right:5px;margin-top:-2px;" alt="Arizona Coyotes" src="https://capfriendly-wlb8ng5.stackpathdns.com/assets/images/logos/arizona_coyotes.svg"><a href="/teams/coyotes">Arizona Coyotes</a></span><div class="r"><a title="Daily Cap Tracker" href="/teams/coyotes/cap-tracker"><img title="Daily Tracker" alt="Daily Tracker" style="margin-bottom:-3px;width:15px;height:15px" width="13" height="15" src="https://capfriendly-wlb8ng5.stackpathdns.com/assets/images/icons/teams/cal.svg" onerror="this.src='https://capfriendly-wlb8ng5.stackpathdns.com/assets/images/icons/teams/cal.png';this.o

In [10]:
tr.find('td', {'data-label': 'FINAL CAP HIT'}, strict=True).text

'$85,089,798'

In [11]:
def parse_tr(tr):
    team = tr.find('a', mode='first').text
    cap = tr.find('td', {'data-label': 'FINAL CAP HIT'}, strict=True).text
    cap = float(cap.replace(',', '').replace('$', ''))
    return team, cap

In [12]:
cap_hits = [parse_tr(tr) for tr in trs]

In [13]:
url = 'https://www.hockey-reference.com/leagues/NHL_2020_standings.html'
html = get(url)

In [14]:
soup = Soup(html)

In [15]:
import pandas as pd

In [16]:
east = pd.read_html(str(soup.find('table')[0]))[0]
west = pd.read_html(str(soup.find('table')[1]))[0]

In [17]:
df = pd.concat([east, west])[['Unnamed: 0', 'GP', 'W']].reset_index(drop=True)

In [18]:
df['W'] = df['W'].apply(pd.to_numeric, errors='coerce')
wins = df.dropna()
wins = wins.rename(columns={'Unnamed: 0': 'Team'})

In [19]:
cap_hits = pd.DataFrame(cap_hits, columns=['Team', 'spend'])
df = pd.merge(wins, cap_hits, on='Team', how='left')
df['mpw'] = round(df['spend'] / df['W'] / 1_000_000, 2)
df.sort_values('mpw', ascending=True)

,Team,GP,W,spend,mpw
17,Colorado Avalanche,70,42.0,76067463.0,1.81
0,Boston Bruins,70,44.0,81653080.0,1.86
1,Tampa Bay Lightning,70,43.0,80238349.0,1.87
16,St. Louis Blues,71,42.0,83216941.0,1.98
8,Washington Capitals,69,41.0,81564505.0,1.99
9,Philadelphia Flyers,69,41.0,82693252.0,2.02
10,Pittsburgh Penguins,69,40.0,81786224.0,2.04
19,Winnipeg Jets,71,37.0,77777881.0,2.10
14,New York Rangers,70,37.0,77613415.0,2.10
23,Vegas Golden Knights,71,39.0,81707634.0,2.10


In [20]:
from IPython.display import HTML
HTML('<img src="https://media.giphy.com/media/oOX5qIDkzDjeo/giphy.gif">')

### Saving results

In [21]:
df.to_csv('../data/mpw.csv', index=False)

In [22]:
df.head()

,Team,GP,W,spend,mpw
0,Boston Bruins,70,44.0,81653080.0,1.86
1,Tampa Bay Lightning,70,43.0,80238349.0,1.87
2,Toronto Maple Leafs,70,36.0,95178332.0,2.64
3,Florida Panthers,69,35.0,81225935.0,2.32
4,Montreal Canadiens,71,31.0,76960206.0,2.48


In [23]:
df['date_fetched'] = pd.Timestamp('today')

In [24]:
df.head()

,Team,GP,W,spend,mpw,date_fetched
0,Boston Bruins,70,44.0,81653080.0,1.86,2020-08-12 13:00:12.128263
1,Tampa Bay Lightning,70,43.0,80238349.0,1.87,2020-08-12 13:00:12.128263
2,Toronto Maple Leafs,70,36.0,95178332.0,2.64,2020-08-12 13:00:12.128263
3,Florida Panthers,69,35.0,81225935.0,2.32,2020-08-12 13:00:12.128263
4,Montreal Canadiens,71,31.0,76960206.0,2.48,2020-08-12 13:00:12.128263


In [25]:
import sqlite3

con = sqlite3.connect('../data/mpw.db')

df.to_sql('teams', con, index=False, if_exists='append')

In [26]:
pd.read_sql('''
    select 
    * 
    from teams 
    where mpw > 2 and W < 41
    order by mpw desc
''', con)

,Team,GP,W,spend,mpw,date_fetched
0,Detroit Red Wings,71,17.0,79968736.0,4.70,2020-08-11 15:14:34.722507
1,Detroit Red Wings,71,17.0,79968736.0,4.70,2020-08-12 13:00:12.128263
2,Ottawa Senators,71,25.0,74319369.0,2.97,2020-08-11 15:14:34.722507
3,Ottawa Senators,71,25.0,74319369.0,2.97,2020-08-12 13:00:12.128263
4,San Jose Sharks,70,29.0,81010454.0,2.79,2020-08-11 15:14:34.722507
5,San Jose Sharks,70,29.0,81010454.0,2.79,2020-08-12 13:00:12.128263
6,Buffalo Sabres,69,30.0,82803567.0,2.76,2020-08-11 15:14:34.722507
7,Buffalo Sabres,69,30.0,82803567.0,2.76,2020-08-12 13:00:12.128263
8,Anaheim Ducks,71,29.0,78692637.0,2.71,2020-08-11 15:14:34.722507
9,Anaheim Ducks,71,29.0,78692637.0,2.71,2020-08-12 13:00:12.128263
